<a href="https://colab.research.google.com/github/tajuar-akash-hub/Deep-learning/blob/main/backprop_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

# ----------------------------
# 1. Dataset (Pure PyTorch)
# ----------------------------
# Format: [Study Hours, Sleep Hours, HSC marks]
import torch

data = torch.tensor([
    [2., 3., 3.],
    [5., 6., 4.],
    [7., 8., 5.]
])

X_all = data[:, 0:2] # Features
y_all = data[:, 2]   # Target

# ----------------------------
# 2. Initialize Parameters
# ----------------------------
def initialize_parameters(layer_dims):
    torch.manual_seed(3)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        # We use .clone() and * 0.1 to match your specific manual init logic
        parameters['W' + str(l)] = torch.ones((layer_dims[l-1], layer_dims[l])) * 0.1
        parameters['b' + str(l)] = torch.zeros((layer_dims[l], 1))
    return parameters

# ----------------------------
# 3. Forward Pass Logic
# ----------------------------
def linear_forward(A_prev, W, b):
    # W.T @ A_prev matches your np.dot(W.T, A_prev)
    Z = torch.matmul(W.T, A_prev) + b
    return Z

def L_layer_forward(X, parameters):
    A = X
    L = len(parameters) // 2
    for l in range(1, L + 1):
        A_prev = A
        Wl = parameters['W' + str(l)]
        bl = parameters['b' + str(l)]
        A = linear_forward(A_prev, Wl, bl)

    # Returning A (y_hat) and A_prev (A1) specifically for your update function
    return A, A_prev

# ---------------------------------------------------
# 4. Manual Update Logic
# ----------------------------------------------------
def update_parameters(parameters, y, y_hat, A1, X, lr=0.001):
    # Your manual backprop logic preserved exactly
    # error = 2 * (y - y_hat)
    err_signal = 2 * (y - y_hat)

     #  Save OLD W2 values before updating

    W2_00_old = parameters['W2'][0, 0].item()
    W2_10_old = parameters['W2'][1, 0].item()

    # Update Layer 2
    parameters['W2'][0, 0] += lr * err_signal * A1[0, 0]
    parameters['W2'][1, 0] += lr * err_signal * A1[1, 0]
    parameters['b2'][0, 0] += lr * err_signal # Preserving your specific b2 logic

    # Update Layer 1 (Using current W2 as per your original code)
    parameters['W1'][0, 0] += lr * err_signal * parameters['W2'][0, 0] * X[0, 0]
    parameters['W1'][0, 1] += lr * err_signal * parameters['W2'][0, 0] * X[1, 0]
    parameters['b1'][0, 0] += lr * err_signal * parameters['W2'][0, 0]

    parameters['W1'][1, 0] += lr * err_signal * parameters['W2'][1, 0] * X[0, 0]
    parameters['W1'][1, 1] += lr * err_signal * parameters['W2'][1, 0] * X[1, 0]
    parameters['b1'][1, 0] += lr * err_signal * parameters['W2'][1, 0]

    return parameters

# ----------------------------
# 5. Training Loop
# ----------------------------
params = initialize_parameters([2, 2, 1])
epochs = 5

for i in range(epochs):
    epoch_loss = 0
    for j in range(X_all.shape[0]):
        # Prepare sample
        X_sample = X_all[j].reshape(2, 1)
        y_sample = y_all[j]

        # Forward
        y_hat, A1 = L_layer_forward(X_sample, params)
        y_hat_scalar = y_hat[0, 0]

        # Update
        params = update_parameters(params, y_sample, y_hat_scalar, A1, X_sample)

        # Loss calculation
        loss = (y_sample - y_hat_scalar) ** 2
        epoch_loss += loss.item()

    print(f"Epoch - {i+1} Loss - {epoch_loss / X_all.shape[0]}")

Epoch - 1 Loss - 14.67866039276123
Epoch - 2 Loss - 13.701451301574707
Epoch - 3 Loss - 12.494118690490723
Epoch - 4 Loss - 11.040447076161703
Epoch - 5 Loss - 9.366330782572428
